### Import modules

### 1. Import the data and Splitting

In [ ]:
import random
import os
import shutil
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder

random.seed(30)

#Path to dataset 
#TBD
data_dir = 'TBD'

#Get all individuals 
individuals = [name for name in os.listdir(data_dir) 
               if os.path.isdir(os.path.join(data_dir, name))]

#Split individuals into train, val, test sets 
#(30% random for combined validation and test set)
train_individuals, test_individuals = train_test_split(individuals, 
                                                       test_size=0.30, 
                                                       random_state=30)
#(50% random for validation and 50 for test set)
val_individuals, test_individuals = train_test_split(individuals, 
                                                     test_size=0.50, 
                                                     random_state=30)


#create directories for the splits (TBD)
train_dir = 'TBD'
val_dir = 'TBD'
test_dir = 'TBD'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

#Define a function to copy images to new directories
def copy_images(individuals, source_dir, dest_dir):
    for individual in individuals:
        individual_dir = os.path.join(source_dir, individual)
        if os.path.isdir(individual_dir):
            shutil.copytree(individual_dir, os.path.join(dest_dir, 
                                                         individual), 
                                                         dirs_exist_ok=True)

copy_images(train_individuals, data_dir, train_dir)
copy_images(val_individuals, data_dir, val_dir)
copy_images(test_individuals, data_dir, test_dir)

#Load dataset via ImageFolder
train_dataset = ImageFolder(train_dir)
val_dataset = ImageFolder(val_dir)
test_dataset = ImageFolder(test_dir)

#Check output of each set
print(f"Number of training images: ", len(train_dataset))
print(f"Number of validation images: ", len(val_dataset))
print(f"Number of test images: ", len(test_dataset)) 

### 2. Build CNN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class YogaClassfier(nn.Module):
    def __init__(self, num_classes = 9):
        super(YogaClassfier, self).__init__()

        #Convolutional layers (increasing numbers of filters)
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels=64, 
                               kernel_size = 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 64, out_channels=128, 
                               kernel_size = 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 128, out_channels=256, 
                               kernel_size = 3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels = 256, out_channels=512, 
                               kernel_size = 3, stride=1, padding=1)
        
        #Max-pooling layers 
        #reducing spatial dimensinons by half after each layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        #Fully connected layers
        self.fc1 = nn.Linear(in_features=512 * 14 * 14, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=num_classes)

    def forward(self, x):
        #Convlutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        #Flatten the feature map
        x = x.view(-1, 512*14*14)

        #Fully connected layers with ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) #handled by loss fuction 

        return x

model = YogaClassfier(num_classes=9)

### 3. Train CNN

### 4. Sanity check

### 5. Hyperparameter Tuning

### 6. Tranfer Learning